<a id=top></a>

# **目次**

<b>
    <details>
        <summary>
            <a href="#modules", style="font-size: xx-large">1. モジュールインポート</a>
            <ul>※サードパーティライブラリ>>>自作モジュール>>>（ここまで本ipynb外）>>>自作関数（本ipynb内）</ul>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#data", style="font-size: xx-large">2. オリジナルデータインポート</a>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#patentcount", style="font-size: xx-large">3. 特許数</a>
        </summary>
        <table></table>
    </details>
    <details>
        <summary>
            <a href="#calculateindicator", style="font-size: xx-large">4. 各指標</a>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#output", style="font-size: xx-large">5. ファイルに出力</a>
        </summary>
    </details>
</b>


---


<a id=modules></a>

## **1. モジュールインポート**


In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../src')
from ecomplexity import ecomplexity

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.3f}'.format


In [2]:
import initial_condition


In [34]:
global data_dir, output_dir
data_dir = '../../data/interim/internal/filtered_after_agg/'
output_dir = '../../data/processed/internal/'



In [3]:
# 初期条件
ar = initial_condition.AR
year_style = initial_condition.YEAR_STYLE

year_start = initial_condition.YEAR_START
year_end = initial_condition.YEAR_END
year_range = initial_condition.YEAR_RANGE

classification = initial_condition.CLASSIFICATION
class_weight = initial_condition.CLASS_WEIGHT
applicant_weight = initial_condition.APPLICANT_WEIGHT

extract_population = initial_condition.EXTRACT_POPULATION
top_p_or_num = initial_condition.TOP_P_OR_NUM


In [2]:
# 自作モジュールインポート
from Scripts.kci import df_to_kci as dtk
from Scripts.vizualize import rank as vr
from Scripts.vizualize import distribution as vd


In [27]:
def kh_ki(c_df, classification, n=19):
    kh1_ki1_df = pd.merge(c_df.copy(), 
                        c_df[c_df['mcp']==1].groupby(['right_person_name'])[['ubiquity']].sum().reset_index(drop=False).copy().rename(columns={'ubiquity':'kh_1'}), 
                        on=['right_person_name'], how='left')
    kh1_ki1_df = pd.merge(kh1_ki1_df.copy(), 
                        c_df[c_df['mcp']==1].groupby([classification])[['diversity']].sum().reset_index(drop=False).copy().rename(columns={'diversity':'ki_1'}), 
                        on=[classification], how='left')
    kh1_ki1_df['kh_1'] = kh1_ki1_df['kh_1'] / kh1_ki1_df['diversity']
    kh1_ki1_df['ki_1'] = kh1_ki1_df['ki_1'] / kh1_ki1_df['ubiquity']
    kh_ki_df = kh1_ki1_df.copy()
    for i in range(n):
        kh_ki_df = pd.merge(kh_ki_df, 
                            kh_ki_df[kh_ki_df['mcp']==1].groupby(['right_person_name'])[[f'ki_{i+1}']].sum().reset_index(drop=False).copy()\
                                        .rename(columns={f'ki_{i+1}':f'kh_{i+2}'}), 
                            on=['right_person_name'], how='left')
        kh_ki_df = pd.merge(kh_ki_df, 
                            kh_ki_df[kh_ki_df['mcp']==1].groupby([classification])[[f'kh_{i+1}']].sum().reset_index(drop=False).copy()\
                                        .rename(columns={f'kh_{i+1}':f'ki_{i+2}'}), 
                            on=[classification], how='left')
        kh_ki_df[f'kh_{i+2}'] = kh_ki_df[f'kh_{i+2}'] / kh_ki_df['diversity']
        kh_ki_df[f'ki_{i+2}'] = kh_ki_df[f'ki_{i+2}'] / kh_ki_df['ubiquity']
    return kh_ki_df


---


<a id=data></a>

## **2. オリジナルデータインポート**


In [5]:
reg_num_top_df = pd.read_csv(f'{data_dir}{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}.csv', 
                             encoding='utf-8',
                             sep=',')
reg_num_top_df

,app_year_period,right_person_name,schmoch_35,reg_num
0,1981-2010,キヤノン株式会社,9,23715.000
1,1981-2010,キヤノン株式会社,28,9984.333
2,1981-2010,キヤノン株式会社,2,9520.833
3,1981-2010,キヤノン株式会社,6,7437.333
4,1981-2010,キヤノン株式会社,3,6348.500
...,...,...,...,...
11673,2001-2010,国立研究開発法人産業技術総合研究所,4,19.833
11674,2001-2010,国立研究開発法人産業技術総合研究所,18,15.583
11675,2001-2010,国立研究開発法人産業技術総合研究所,34,14.667
11676,2001-2010,国立研究開発法人産業技術総合研究所,7,11.000


<a href=#top>先頭に戻る</a>

---


<a id=calculateindicator></a>

## **4. 各指標**


In [12]:
trade_cols = {'time':f'{ar}_{year_style}_period', 'loc':'right_person_name', 'prod':classification, 'val':'reg_num'}
rename_col_dict = {'eci':'kci', 'pci':'tci'}
col_order_list = [f'{ar}_{year_style}_period', 'right_person_name', classification, 'reg_num', 'rca', 'mcp', 'diversity', 'ubiquity', 'kci', 'tci']


In [35]:
c_df = ecomplexity(reg_num_top_df,
                   cols_input = trade_cols, 
                   rca_mcp_threshold = 1)
c_df = c_df[c_df['reg_num'] > 0]\
           .rename(columns=rename_col_dict)\
           [col_order_list]
c_df = pd.concat([kh_ki(c_df[c_df[f'{ar}_{year_style}_period'] == period], classification) for period in c_df[f'{ar}_{year_style}_period'].unique()], 
                 axis='index', 
                 ignore_index=True)

# for segment in c_df[f'{ar}_{year_style}_period'].unique():
#     display(c_df[c_df[f'{ar}_{year_style}_period'] == segment].head())
#     display(c_df[c_df[f'{ar}_{year_style}_period'] == segment].describe())
#     print(c_df[c_df[f'{ar}_{year_style}_period'] == segment].info())
#     print('\n')


1981-2010
1981-1990
1991-2000
2001-2010


In [28]:
c_df.sort_values(by=[f'{ar}_{year_style}_period', 'kci'], ascending=[True, False])

,app_year_period,right_person_name,schmoch_35,reg_num,rca,mcp,diversity,ubiquity,kci,tci,...,kh_17,ki_17,kh_18,ki_18,kh_19,ki_19,kh_20,ki_20,kh_21,ki_21
4105,1981-1990,住友化学株式会社,1,15.000,0.062,0,8,23,1.344,-0.620,...,25.317,9.291,9.316,25.246,25.296,9.296,9.310,25.254,25.284,9.298
4106,1981-1990,住友化学株式会社,2,12.500,0.046,0,8,20,1.344,-2.050,...,25.317,9.275,9.316,25.211,25.296,9.286,9.310,25.233,25.284,9.292
4107,1981-1990,住友化学株式会社,6,0.500,0.002,0,8,15,1.344,-2.554,...,25.317,9.269,9.316,25.198,25.296,9.282,9.310,25.225,25.284,9.290
4108,1981-1990,住友化学株式会社,8,10.000,0.055,0,8,20,1.344,-1.902,...,25.317,9.277,9.316,25.214,25.296,9.287,9.310,25.235,25.284,9.293
4109,1981-1990,住友化学株式会社,9,135.000,0.480,0,8,21,1.344,-0.908,...,25.317,9.289,9.316,25.239,25.296,9.294,9.310,25.250,25.284,9.297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10769,2001-2010,東芝テック株式会社,31,1.000,0.027,0,6,24,-1.697,1.315,...,26.037,9.518,9.510,26.052,26.043,9.517,9.513,26.051,26.045,9.517
10770,2001-2010,東芝テック株式会社,32,4.000,0.035,0,6,19,-1.697,1.335,...,26.037,9.518,9.510,26.052,26.043,9.517,9.513,26.051,26.045,9.517
10771,2001-2010,東芝テック株式会社,33,21.000,0.406,0,6,10,-1.697,-0.430,...,26.037,9.513,9.510,26.046,26.043,9.514,9.513,26.047,26.045,9.515
10772,2001-2010,東芝テック株式会社,34,19.500,0.777,0,6,19,-1.697,-0.582,...,26.037,9.514,9.510,26.046,26.043,9.515,9.513,26.047,26.045,9.515


<a href="#top">先頭に戻る</a>

---


<a id=output></a>

## **5. ファイルに出力**

<a id=rightperson></a>

### **5.1. 特許権者**


In [36]:

right_person_df = pd.merge(c_df.groupby([f'{ar}_{year_style}_period', 'right_person_name'])[['reg_num']].sum().reset_index(drop=False), 
                           c_df.groupby([f'{ar}_{year_style}_period', 'right_person_name'])[[classification]].nunique().reset_index(drop=False), 
                           on=[f'{ar}_{year_style}_period', 'right_person_name'], 
                           how='inner')
right_person_df = pd.merge(right_person_df, 
                           c_df[[f'{ar}_{year_style}_period', 'right_person_name', 'diversity', 'kci']\
                               +[f'kh_{i}' for i in range(1, 20+1)]]\
                               .drop_duplicates(keep='first'), 
                           on=[f'{ar}_{year_style}_period', 'right_person_name'], 
                           how='inner')
# for period in right_person_df[f'{ar}_{year_style}_period'].unique():
#     for i in range(1, 20+1):
#         value = right_person_df[right_person_df[f'{ar}_{year_style}_period']==period]
#         right_person_df[right_person_df[f'{ar}_{year_style}_period']==period][f'kh_{i}'] = (value[f'kh_{i}'] - value[f'kh_{i}'].mean()) / value[f'kh_{i}'].std()
#     display(right_person_df[right_person_df[f'{ar}_{year_style}_period'] == period].head())
#     display(right_person_df[right_person_df[f'{ar}_{year_style}_period'] == period].describe())
#     print(right_person_df[right_person_df[f'{ar}_{year_style}_period'] == period].info())
#     print('\n')
# right_person_df['reg_num'] = right_person_df['reg_num'].astype(np.int64)

In [37]:
right_person_df.to_csv(f'{output_dir}firms/{ar}_{year_style}_{top_p_or_num[0]}_{top_p_or_num[1]}.csv', 
                       encoding='utf-8', 
                       sep=',', 
                       index=False)


<a href=#top>先頭に戻る</a>

---


<a id=ipc></a>

### **5.2. IPC**


In [39]:
# 各期間
classification_df = pd.merge(c_df.groupby([f'{ar}_{year_style}_period', classification])[['reg_num']].sum().reset_index(drop=False), 
                        c_df.groupby([f'{ar}_{year_style}_period', classification])[['right_person_name']].nunique().reset_index(drop=False), 
                        on=[f'{ar}_{year_style}_period', classification], 
                        how='inner')
classification_df = pd.merge(classification_df, 
                      c_df[[f'{ar}_{year_style}_period', classification, 'ubiquity', 'tci']\
                          +[f'ki_{i}' for i in range(1, 20+1)]]\
                          .drop_duplicates(keep='first'), 
                      on=[f'{ar}_{year_style}_period', classification], 
                      how='inner')
classification_df['reg_num'] = classification_df['reg_num'].astype(np.int64)
display(classification_df)


,app_year_period,schmoch_35,reg_num,right_person_name,ubiquity,tci,ki_1,ki_2,ki_3,ki_4,...,ki_11,ki_12,ki_13,ki_14,ki_15,ki_16,ki_17,ki_18,ki_19,ki_20
0,1981-1990,1,34703,93,23,-0.620,8.565,24.518,8.864,24.535,...,9.252,25.166,9.273,25.208,9.285,25.232,9.291,25.246,9.296,25.254
1,1981-1990,2,38474,93,20,-2.050,7.150,21.825,8.301,23.174,...,9.179,25.004,9.228,25.110,9.258,25.173,9.275,25.211,9.286,25.233
2,1981-1990,3,18335,85,18,-2.221,7.556,21.440,8.249,23.036,...,9.166,24.981,9.221,25.096,9.254,25.165,9.273,25.206,9.284,25.230
3,1981-1990,4,3464,51,9,-2.618,9.444,20.223,8.435,22.554,...,9.145,24.934,9.208,25.068,9.246,25.148,9.268,25.196,9.282,25.224
4,1981-1990,5,9091,67,21,-2.449,7.524,21.068,8.265,22.731,...,9.155,24.954,9.214,25.080,9.249,25.156,9.270,25.200,9.283,25.227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2001-2010,31,11367,95,24,1.315,8.375,25.876,9.237,26.094,...,9.522,26.066,9.521,26.059,9.520,26.055,9.518,26.052,9.517,26.051
136,2001-2010,32,35735,89,19,1.335,7.263,25.197,9.009,25.914,...,9.517,26.064,9.519,26.058,9.519,26.054,9.518,26.052,9.517,26.051
137,2001-2010,33,16080,88,10,-0.430,8.000,21.409,8.915,24.734,...,9.488,26.026,9.502,26.038,9.509,26.044,9.513,26.046,9.514,26.047
138,2001-2010,34,7800,84,19,-0.582,9.053,24.511,9.291,25.593,...,9.501,26.033,9.508,26.040,9.512,26.044,9.514,26.046,9.515,26.047


In [40]:
classification_df.to_csv(f'{output_dir}technology/{ar}_{year_style}_{top_p_or_num[0]}_{top_p_or_num[1]}.csv', 
                        encoding='utf-8', 
                        sep=',', 
                        index=False)


<a href=#top>先頭に戻る</a>

---


<a id=network></a>

## **5.3. 二部グラフ用**


In [16]:
graph_df = pd.concat([c_df, c_df], axis='index')
graph_df = graph_df[graph_df['mcp']==1][[f'{ar}_{year_style}', 'right_person_name', 'ipc_class', 'mcp']]
graph_df

,segment,right_person_name,ipc_class,mcp
0,1981-2010,あすか製薬株式会社,A23D,1
1,1981-2010,あすか製薬株式会社,A23K,1
2,1981-2010,あすか製薬株式会社,A23L,1
3,1981-2010,あすか製薬株式会社,A47C,1
5,1981-2010,あすか製薬株式会社,A61G,1
...,...,...,...,...
58712,2001-2010,Ｚホールデイングス株式会社,G10K,1
58713,2001-2010,Ｚホールデイングス株式会社,G10L,1
58715,2001-2010,Ｚホールデイングス株式会社,H03M,1
58716,2001-2010,Ｚホールデイングス株式会社,H04H,1


In [17]:
graph_df.to_csv(f'../Data/0_Graph/{ar}_{year_start}_{year_end}.csv', 
                encoding='utf-8', 
                sep=',', 
                index=False)
graph_df


,segment,right_person_name,ipc_class,mcp
0,1981-2010,あすか製薬株式会社,A23D,1
1,1981-2010,あすか製薬株式会社,A23K,1
2,1981-2010,あすか製薬株式会社,A23L,1
3,1981-2010,あすか製薬株式会社,A47C,1
5,1981-2010,あすか製薬株式会社,A61G,1
...,...,...,...,...
58712,2001-2010,Ｚホールデイングス株式会社,G10K,1
58713,2001-2010,Ｚホールデイングス株式会社,G10L,1
58715,2001-2010,Ｚホールデイングス株式会社,H03M,1
58716,2001-2010,Ｚホールデイングス株式会社,H04H,1
